In [31]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

import math

## Crane
This file is based on the mass_spring.ipynb file. It was modified to model a crane

In [32]:
mss = MassSpringSystem3d()
mss.gravity = (0,-9.81,0) #gravity in comparison to mass_spring in other direction!

distance = 1./math.sqrt(2.)
stiffGlob = 20000

f1 = mss.add (Fix( (0,0,0)))
f2 = mss.add (Fix( (1,0,0)))
f3 = mss.add (Fix( (0.5,distance,0)))
#mss.add (Spring(1, stiffGlob, (f1, f2)))
#mss.add (Spring(1, stiffGlob, (f2, f3)))
#mss.add (Spring(1, stiffGlob, (f3, f1)))

for i in range(4):
    name = "line" + str(i)
    z_pos = 0.7 * (i+1)
    ExecStringA = "mA_" + name + " = mss.add (Mass(0.1, (0,0,z_pos)))"
    ExecStringB = "mB_" + name + " = mss.add (Mass(0.1, (1,0,z_pos)))"
    ExecStringC = "mC_" + name + " = mss.add (Mass(0.1, (0.5,distance,z_pos)))"
    exec(ExecStringA)
    exec(ExecStringB)
    exec(ExecStringC)
    ExecString_Springs = """mss.add (Spring(1, stiffGlob, """
    ExecString_Springs += "(mA_" + str(name) + ",mB_" + str(name) + "))) \n"
    ExecString_Springs += "mss.add (Spring(1, stiffGlob, (mB_"+ str(name) + ",mC_" + str(name) + "))) \n"
    ExecString_Springs += "mss.add (Spring(1, stiffGlob, (mC_"+ str(name) + ",mA_" + str(name) + ")))"
    exec(ExecString_Springs)
    if i == 0:
        mss.add (Spring(1, stiffGlob, (mA_line0, f1)))
        mss.add (Spring(1, stiffGlob, (mB_line0, f2)))
        mss.add (Spring(1, stiffGlob, (mC_line0, f3)))
        mss.add (Spring(1, stiffGlob, (f3, mA_line0)))
        mss.add (Spring(1, stiffGlob, (f3, mB_line0)))
        mss.add (Spring(1, stiffGlob, (f2, mA_line0)))
    else:
        name_old = "line" + str(i-1)
        ExecString_connect = """mss.add (Spring(1, stiffGlob, ("""
        ExecString_connect += "mA_" + name + ", mA_" + name_old + "))) \n"
        ExecString_connect += "mss.add (Spring(1, stiffGlob, (mB_" + name + ", mB_" + name_old + "))) \n"
        ExecString_connect += "mss.add (Spring(1, stiffGlob, (mC_" + name + ", mC_" + name_old + "))) \n"
        ExecString_connect += "mss.add (Spring(1, stiffGlob, (mC_" + name_old + ", mA_" + name + "))) \n"
        ExecString_connect += "mss.add (Spring(1, stiffGlob, (mC_" + name_old + ", mB_" + name + "))) \n"
        ExecString_connect += "mss.add (Spring(1, stiffGlob, (mB_" + name_old + ", mA_" + name + ")))"
        exec(ExecString_connect)

mDyn = mss.add (Mass(4, (2,0.5,z_pos)))
mss.add (Spring(1, stiffGlob, (mC_line3, mDyn)))



36

In [33]:
masses = []
for m in mss.masses[0:-1]: # append everything except the vibration inducing mass
    masses.append(Mesh(SphereBufferGeometry(0.2, 16, 16), MeshStandardMaterial(color='red'), position=m.pos)) 

masses.append(Mesh(SphereBufferGeometry(0.2, 16, 16), MeshStandardMaterial(color='green'), position=mss.masses[-1].pos))
mass_object = mss.masses[-1]
mass_object.vel = (0.0 ,0.0, 200.0) # setting the last mass with an inital velocity

fixes = []
for f in mss.fixes:
    fixes.append(Mesh(SphereBufferGeometry(0.2, 32, 16), MeshStandardMaterial(color='blue'), position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeom = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan') # only visible in longr chain elements
springs = LineSegments2(springgeom, m2)    

axes = AxesHelper(1)

In [34]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height, up=[0,0,1])
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [35]:
from time import sleep
for i in range(10000):
    mss.simulate (0.001, 1000)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

ValueError: Newton did not converge